### Code to Create 50 Week Heatmap With Time Slider
This must be run on Compute Canada. Data is big. It can be found in the remote directory:
sahakhsh@cedar.computecanada.ca:~/scratch/50week_Heatmap_with_TimeSlider

#### Import Libraries

In [2]:
import os
import pandas as pd
import ast
import datetime
import folium
from folium import plugins
from folium.plugins import HeatMap

#### Create places dataframe

In [34]:
with open('./covid_19_tweets/places.txt') as f:
    places = []
    for line in f:
        tempdict = {}
        line = ast.literal_eval(line)
        bottomleft_long = line["geo"]["bbox"][0]
        bottomleft_lat = line["geo"]["bbox"][1]
        topright_long = line["geo"]["bbox"][2]
        topright_lat = line["geo"]["bbox"][3]
        tempdict["id"] = line["id"]
        tempdict["latitude"] = (bottomleft_lat + topright_lat)/2
        tempdict["longitude"] = (bottomleft_long + topright_long)/2
        places.append(tempdict)
    places_df = pd.DataFrame(places, columns = ['id', 'latitude', 'longitude']) 
    print(places_df)

                      id   latitude   longitude
0       b7d3c12268abd20e  51.238447    6.814308
1       01a9a39529b27f36  40.780709  -73.968542
2       e86b380cfefcced5  41.500753  -72.757385
3       317fcc4b21a604d5  28.638489   77.095086
4       813a485b26b8dae2  35.117003 -106.632718
...                  ...        ...         ...
118818  07d9e8a713086002  12.968259   79.948975
118819  0909a45fdf54c001  29.674320   77.846767
118820  07d9cc9aef083003   0.216096   37.798956
118821  07d9cb7e4d885000  52.500561   -2.131352
118822  07d9f79417085001 -25.806896   28.091190

[118823 rows x 3 columns]


#### Dump and Save Contents of Geotweet Files as CSV File

In [35]:
fetch_dir = r'/home/sahakhsh/scratch/covid_19_geotweets/'
dump_dir = r'/home/sahakhsh/scratch/covid_19_geotweets/savedDFs/'
i=0
for filename in os.listdir(fetch_dir):
    geofilename = filename[:-4] + r'.csv'
    if filename.endswith("txt") and geofilename not in os.listdir(dump_dir):
        fetch_path = os.path.join(fetch_dir, filename)
        print(fetch_path)
        with open(fetch_path, encoding='utf-8') as json_file:
            geotweets = []
            for line in json_file:
                geotweets.append(ast.literal_eval(line))
        geotweet_df = pd.DataFrame(geotweets, columns = ['possibly_sensitive', 'entities', 'text', 'public_metrics', 'id', 'context_annotations', 'lang', 'in_reply_to_user_id', 'author_id', 'source', 'referenced_tweets', 'created_at', 'geo']) 
        geotweet_df.to_csv(os.path.join(dump_dir+geofilename))
    print(filename)
    i=i+1

# Test to see if files were saved successfully
# mydf = pd.read_csv('/home/sahakhsh/scratch/covid_19_geotweets/savedDFs/corona_geotweets_350_data.csv', lineterminator='\n')
# mydf

corona_geotweets_152_data.txt
corona_geotweets_205_data.txt
corona_geotweets_275_data.txt
corona_geotweets_321_data.txt
corona_geotweets_32_data.txt
corona_geotweets_246_data.txt
corona_geotweets_305_data.txt
corona_geotweets_52_data.txt
corona_geotweets_10_data.txt
corona_geotweets_328_data.txt
corona_geotweets_243_data.txt
corona_geotweets_68_data.txt
corona_geotweets_265_data.txt
corona_geotweets_93_data.txt
corona_geotweets_331_data.txt
corona_geotweets_89_data.txt
corona_geotweets_309_data.txt
corona_geotweets_58_data.txt
corona_geotweets_234_data.txt
corona_geotweets_101_data.txt
corona_geotweets_04_data.txt
corona_geotweets_63_data.txt
corona_geotweets_13_data.txt
corona_geotweets_196_data.txt
corona_geotweets_07_data.txt
corona_geotweets_272_data.txt
corona_geotweets_117_data.txt
corona_geotweets_207_data.txt
corona_geotweets_177_data.txt
corona_geotweets_15_data.txt
corona_geotweets_30_data.txt
corona_geotweets_261_data.txt
corona_geotweets_252_data.txt
corona_geotweets_226_da

#### Process Geotweet CSV files

In [73]:
# Redo all....
# Read each csv file, convert to df, process by adding relevant columns and merging with places dataframe
# actually instead of redoing, keep these, dump processed ones under saved_processed_dfs
# tws_merge = pd.merge(tws_df,pls_df,how = 'inner',left_on = 'geo_id',right_on = 'id')
fetch_dir = r'/home/sahakhsh/scratch/covid_19_geotweets/savedDFs/'
dump_dir = r'/home/sahakhsh/scratch/covid_19_geotweets/processed_geotweets/'

def get_hour(d):
    new_date = datetime.datetime.strptime(d,"%Y-%m-%dT%H:%M:%S.%fZ")
    return new_date.hour

def get_referenced_ids(referenced_tweets):
    refer_tweets = []
    if not type(referenced_tweets) == type(1.23):
        referenced_tweets = ast.literal_eval(referenced_tweets)
    if type(referenced_tweets) == type([]) and referenced_tweets:
        for reference in referenced_tweets:
            refer_tweets.append(reference['id'])
    return refer_tweets

def get_mentions(entities):
    users_mentioned = []
    if type(entities) == type(""):
        entities = ast.literal_eval(entities)
    if type(entities) == type({}) and "mentions" in entities:
        for user in entities["mentions"]:
            users_mentioned.append(user['username'])
    return users_mentioned

def extract_place(geo):
    if type(geo) == type({}) and "place_id" in geo:
        return geo["place_id"]

for filename in os.listdir(fetch_dir):
    print(filename)
    geotweet_df = pd.read_csv(os.path.join(fetch_dir, filename), lineterminator='\n') 
    geotweet_df['hour_tweeted'] = geotweet_df['created_at'].apply(get_hour)
    geotweet_df['referenced_tweet_ids'] = geotweet_df['referenced_tweets'].apply(get_referenced_ids)
    geotweet_df["users_mentioned"] = geotweet_df["entities"].apply(get_mentions)
    geotweet_df['place_id'] = geotweet_df['geo'].map(ast.literal_eval).apply(extract_place)
    df_merged = pd.merge(geotweet_df,places_df,how = 'inner',left_on = 'place_id',right_on = 'id')
    df_merged.to_csv(os.path.join(dump_dir, filename))

corona_geotweets_129_data.csv
corona_geotweets_319_data.csv
corona_geotweets_49_data.csv
corona_geotweets_158_data.csv
corona_geotweets_164_data.csv
corona_geotweets_128_data.csv
corona_geotweets_350_data.csv
corona_geotweets_328_data.csv
corona_geotweets_335_data.csv
corona_geotweets_230_data.csv
corona_geotweets_312_data.csv
corona_geotweets_106_data.csv
corona_geotweets_120_data.csv
corona_geotweets_136_data.csv
corona_geotweets_220_data.csv
corona_geotweets_22_data.csv
corona_geotweets_113_data.csv
corona_geotweets_53_data.csv
corona_geotweets_299_data.csv
corona_geotweets_150_data.csv
corona_geotweets_34_data.csv
corona_geotweets_277_data.csv
corona_geotweets_279_data.csv
corona_geotweets_346_data.csv
corona_geotweets_165_data.csv
corona_geotweets_48_data.csv
corona_geotweets_08_data.csv
corona_geotweets_59_data.csv
corona_geotweets_81_data.csv
corona_geotweets_114_data.csv
corona_geotweets_286_data.csv
corona_geotweets_100_data.csv
corona_geotweets_84_data.csv
corona_geotweets_47

corona_geotweets_239_data.csv
corona_geotweets_303_data.csv
corona_geotweets_256_data.csv
corona_geotweets_122_data.csv
corona_geotweets_15_data.csv
corona_geotweets_253_data.csv
corona_geotweets_275_data.csv
corona_geotweets_244_data.csv
corona_geotweets_297_data.csv
corona_geotweets_153_data.csv
corona_geotweets_344_data.csv
corona_geotweets_264_data.csv
corona_geotweets_160_data.csv
corona_geotweets_269_data.csv
corona_geotweets_29_data.csv
corona_geotweets_38_data.csv
corona_geotweets_46_data.csv
corona_geotweets_27_data.csv
corona_geotweets_50_data.csv
corona_geotweets_234_data.csv
corona_geotweets_237_data.csv
corona_geotweets_66_data.csv
corona_geotweets_321_data.csv
corona_geotweets_213_data.csv
corona_geotweets_304_data.csv
corona_geotweets_341_data.csv
corona_geotweets_62_data.csv
corona_geotweets_161_data.csv
corona_geotweets_64_data.csv
corona_geotweets_214_data.csv
corona_geotweets_301_data.csv
corona_geotweets_317_data.csv
corona_geotweets_163_data.csv
corona_geotweets_70

#### Create List of Geotweet Dataframes by Week

In [77]:
i = 1
week = 0
merged = [[]]
fetch_dir = r'/home/sahakhsh/scratch/covid_19_geotweets/processed_geotweets/'
while i < 351:
    if i<10:
        filename = 'corona_geotweets_0'+str(i)+'_data.csv'
    else:
        filename = 'corona_geotweets_'+str(i)+'_data.csv'
    name = os.path.join(fetch_dir, filename)
    temp = pd.read_csv(name, lineterminator='\n')
    merged[week].append(temp)
    i+=1
    if i % 7 == 1:
        merged[week] = pd.concat(merged[week]) 
        week+=1
        merged.append([])
merged.pop(-1)

#### Adjust Merged Dataframes by Week

In [87]:
for element in merged:
    element.pop('id_y')
    element.pop('Unnamed: 0')
    element.pop('Unnamed: 0.1')    

#### Assemble heat map data

In [90]:
heat_data = [[[row['latitude'],row['longitude']] for index, row in merged[i].iterrows()] for i in range(len(merged))]
hm = plugins.HeatMapWithTime(heat_data,auto_play=True,max_opacity=0.8)

NameError: name 'map_hooray' is not defined

#### Add heatmap data to map

In [95]:
map_hooray = folium.Map([37.5, -125], tiles='stamentoner', zoom_start=3)
hm.add_to(map_hooray)

#### Display Heatmap

In [100]:
map_hooray